In [8]:
import glob

path = '/Users/kris/data_sources/fdic/BulkReports/'

In [10]:
for folder in glob.glob(path + '*[!.]'):
    print(f'{folder}')

/Users/kris/data_sources/fdic/BulkReports/BANK OF EASTERN OREGON_14865
/Users/kris/data_sources/fdic/BulkReports/KCB BANK_5751
/Users/kris/data_sources/fdic/BulkReports/PINELAND BANK_6039
/Users/kris/data_sources/fdic/BulkReports/DIME COMMUNITY BANK_23504
/Users/kris/data_sources/fdic/BulkReports/GENOA COMMUNITY BANK_2750
/Users/kris/data_sources/fdic/BulkReports/LAMAR NATIONAL BANK_2161
/Users/kris/data_sources/fdic/BulkReports/PEOPLES BANK THE_7045
/Users/kris/data_sources/fdic/BulkReports/FIRST NATIONAL BANK OF GORDON THE_3252
/Users/kris/data_sources/fdic/BulkReports/WEST UNION BANK_22730
/Users/kris/data_sources/fdic/BulkReports/FNB SOUTH_12030
/Users/kris/data_sources/fdic/BulkReports/FARMERS AND MERCHANTS BANK_4231
/Users/kris/data_sources/fdic/BulkReports/F  C BANK_14155
/Users/kris/data_sources/fdic/BulkReports/BANK7_18854
/Users/kris/data_sources/fdic/BulkReports/BANK OF CLARENDON THE_3720
/Users/kris/data_sources/fdic/BulkReports/HAVERFORD TRUST COMPANY THE_17110
/Users/kris

In [9]:
folders = glob.glob(path + '*[!.]')
print(f'{folders}')

['/Users/kris/data_sources/fdic/BulkReports/BANK OF EASTERN OREGON_14865', '/Users/kris/data_sources/fdic/BulkReports/KCB BANK_5751', '/Users/kris/data_sources/fdic/BulkReports/PINELAND BANK_6039', '/Users/kris/data_sources/fdic/BulkReports/DIME COMMUNITY BANK_23504', '/Users/kris/data_sources/fdic/BulkReports/GENOA COMMUNITY BANK_2750', '/Users/kris/data_sources/fdic/BulkReports/LAMAR NATIONAL BANK_2161', '/Users/kris/data_sources/fdic/BulkReports/PEOPLES BANK THE_7045', '/Users/kris/data_sources/fdic/BulkReports/FIRST NATIONAL BANK OF GORDON THE_3252', '/Users/kris/data_sources/fdic/BulkReports/WEST UNION BANK_22730', '/Users/kris/data_sources/fdic/BulkReports/FNB SOUTH_12030', '/Users/kris/data_sources/fdic/BulkReports/FARMERS AND MERCHANTS BANK_4231', '/Users/kris/data_sources/fdic/BulkReports/F  C BANK_14155', '/Users/kris/data_sources/fdic/BulkReports/BANK7_18854', '/Users/kris/data_sources/fdic/BulkReports/BANK OF CLARENDON THE_3720', '/Users/kris/data_sources/fdic/BulkReports/H

In [2]:
import pandas as pd
import datetime

df = pd.DataFrame({'Date': '20240506', 'As_of': datetime.datetime.now()}, index=[0])
df0 = pd.DataFrame(columns=['Date', 'As_of'])
df1 = pd.DataFrame({'Date': '20240506', 'As_of': datetime.datetime.now()}, index=[0])

#df = pd.concat([df, df1], ignore_index=True)

In [4]:
df0 = pd.concat([df0, df1], ignore_index=True)

In [10]:
df = pd.concat(df1, df1)

,Date,As_of
0,20240506,2024-06-13 11:04:13.840889
1,20240506,2024-06-13 11:04:13.843563


In [19]:
#type(df['Date'])
#df['Date'].isin(['20240506'])

'20240506' in df['Date'].values

True

In [20]:
df.shape[0]

2

In [21]:
skipped_dtypes = {'Date': str, 'As_of': datetime.datetime}
skipped_dtypes.keys()

dict_keys(['Date', 'As_of'])

In [11]:
import time

t = time.time()
period = 60

diff = t - period

current_time = time.strftime('%H:%M:%S', time.localtime(t))
wait_time = time.strftime('%H:%M:%S', time.localtime(diff))

milliseconds_current = int((t - int(t)) * 1000)
milliseconds_diff = int((diff - int(diff)) * 1000)

print(f'current: {current_time}.{milliseconds_current}\nwait: {wait_time}.{milliseconds_diff}\n')


current: 14:51:36.940
wait: 14:50:36.940



In [6]:
import time
import datetime

current = time.time()
#print(datetime.datetime.fromtimestamp(current + 3600).strftime("%Y%m%d %H:%M:%S"))
print(f'{time.strftime("%Y%m%d %H:%M:%S",time.gmtime(current + 3600))}')

20240702 15:07:00


In [9]:
#ij = 0
for i in range(10):
    for j in range(5):
        ij +=1
print(ij)

100
